In [1]:
#Import libraries
import pandas as pd
import requests
import matplotlib.pyplot as plt
import numpy as np
from datetime import date
import matplotlib.ticker as ticker
import plotly.graph_objects as go

In [2]:
# Set up model variables
# API Key from EIA
api_key = 'dc1de54d5800a76b0e84950e9eef64e7'

# PADD Names to Label Columns
# Refiner and Blender Net Input of Crude Oil aka Crude Demand by PADD
PADD_NAMES = ['PADD 1','PADD 2','PADD 3','PADD 4','PADD 5']
PADD_KEY = ['PET.MCRRIP12.M','PET.MCRRIP22.M','PET.MCRRIP32.M','PET.MCRRIP42.M','PET.MCRRIP52.M']

In [3]:
# Initialize list
final_data = []

# Choose start and end dates
startDate = '2015-01-01'
endDate = '2021-01-01'

In [4]:
# Pull in data via EIA API
for i in range(len(PADD_KEY)):
          #http://api.eia.gov/series/?api_key=YOUR_API_KEY_HERE&series_id=PET.MCRRIP32.M
    url = 'http://api.eia.gov/series/?api_key=' + api_key + '&series_id=' + PADD_KEY[i]
    
    r = requests.get(url)
    json_data = r.json()
    
    if r.status_code == 200:
        print('Success!')
    else:
        print('Error')
    
    df = pd.DataFrame(json_data.get('series')[0].get('data'), columns = ['Date', PADD_NAMES[i]])
    df.set_index('Date', drop=True, inplace=True)
    final_data.append(df)

Success!
Success!
Success!
Success!
Success!


In [5]:
# Create final dataframeby concatenating all the df in final_data together
crude = pd.concat(final_data, axis=1)

In [6]:
# Create date as datetype datatype
crude['Year'] = crude.index.astype(str).str[:4]
crude['Month'] = crude.index.astype(str).str[4:]
crude['Day'] = 1
crude['Date'] = pd.to_datetime(crude[['Year','Month','Day']])
crude.set_index('Date',drop=True,inplace=True)
crude.sort_index(inplace=True)
crude = crude[startDate:endDate]
crude['Total'] = crude['PADD 1'] + crude['PADD 2'] + crude['PADD 3'] + crude['PADD 4'] + crude['PADD 5']

In [7]:
# Pivot Table for PADDs and Total
p1_pivot = pd.pivot_table(crude, values='PADD 1', index=['Month'],
                    columns=['Year'], aggfunc=np.sum)

p2_pivot = pd.pivot_table(crude, values='PADD 2', index=['Month'],
                    columns=['Year'], aggfunc=np.sum)

p3_pivot = pd.pivot_table(crude, values='PADD 3', index=['Month'],
                    columns=['Year'], aggfunc=np.sum)

p4_pivot = pd.pivot_table(crude, values='PADD 4', index=['Month'],
                    columns=['Year'], aggfunc=np.sum)

p5_pivot = pd.pivot_table(crude, values='PADD 5', index=['Month'],
                    columns=['Year'], aggfunc=np.sum)

total_pivot = pd.pivot_table(crude, values='Total', index=['Month'],
                    columns=['Year'], aggfunc=np.sum)

In [8]:
# Add max, min, and 5 Year Avg to each plot
pivot_list = [p1_pivot, p2_pivot, p3_pivot, p4_pivot, p5_pivot, total_pivot]

for pivot_choice in pivot_list:
    pivot_choice['Max'] = pivot_choice.iloc[:,:5].max(axis=1)
    pivot_choice['Min'] = pivot_choice.iloc[:,:5].min(axis=1)
    pivot_choice['5 Year Avg'] = pivot_choice.iloc[:,:5].mean(axis=1)

In [9]:
# INSTALL PLOTLY pip install plotly with anaconda prompt

In [40]:
# SHOWS ALL LINES
# Set up Figure
p1_fig = go.Figure()
p2_fig = go.Figure()
p3_fig = go.Figure()
p4_fig = go.Figure()
p5_fig = go.Figure()
total_fig = go.Figure()

# Set up lists to iterate through
fig_list = [p1_fig, p2_fig, p3_fig, p4_fig, p5_fig, total_fig]
pivot_list = [p1_pivot, p2_pivot, p3_pivot, p4_pivot, p5_pivot, total_pivot]
title_list = ['PADD 1 Crude Demand (kbd)','PADD 2 Crude Demand (kbd)','PADD 3 Crude Demand (kbd)',
              'PADD 4 Crude Demand (kbd)','PADD 5 Crude Demand (kbd)','Total Crude Demand (kbd)']

# Set the colors for the lines
colors = ['#17becf','#e377c2','#ff7f0e','#2ca02c','darkblue','firebrick']

# Create x-axis labels for the months
months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

# Create the Graphs
for j in range(6):
    data = pivot_list[j]

    fig_list[j].add_trace(go.Scatter(x=months, y=data['Max'],
        fill=None,
        mode=None,
        line_color='lightgray', showlegend=False
        ))

    fig_list[j].add_trace(go.Scatter(x=months, y=data['Min'],
        fill='tonexty', # fill area between trace0 and trace1
        mode=None, line_color='lightgray', showlegend=False))

    fig_list[j].add_trace(go.Scatter(x=months, y=data['5 Year Avg'], name='5 Year Avg',
                                 line=dict(color='black', width=4, dash='dot')))

    for i in range(6):
        fig_list[j].add_trace(go.Scatter(x=months, y=data.iloc[:,i], name=data.columns[i],
                                 line=dict(color=colors[i], width=4)))

    fig_list[j].update_layout(title=title_list[j],
                       xaxis_title='Month',
                       yaxis_title='Thousand Barrels/Day', template='plotly_white')
    
# Show all the graphs
p1_fig.show()
p2_fig.show()
p3_fig.show()
p4_fig.show()
p5_fig.show()
total_fig.show()

In [42]:
# SHOWS ONLY THIS YEAR AND LAST YEAR
# Set up Figure
p1_fig = go.Figure()
p2_fig = go.Figure()
p3_fig = go.Figure()
p4_fig = go.Figure()
p5_fig = go.Figure()
total_fig = go.Figure()

# Set up lists to iterate through
fig_list = [p1_fig, p2_fig, p3_fig, p4_fig, p5_fig, total_fig]
pivot_list = [p1_pivot, p2_pivot, p3_pivot, p4_pivot, p5_pivot, total_pivot]
title_list = ['PADD 1 Crude Demand (kbd)','PADD 2 Crude Demand (kbd)','PADD 3 Crude Demand (kbd)',
              'PADD 4 Crude Demand (kbd)','PADD 5 Crude Demand (kbd)','Total Crude Demand (kbd)']

# Set the colors for the lines
colors = ['#17becf','#e377c2','#ff7f0e','#2ca02c','darkblue','firebrick']

# Create x-axis labels for the months
months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

# Create the Graphs
for j in range(6):
    data = pivot_list[j]

    fig_list[j].add_trace(go.Scatter(x=months, y=data['Max'],
        fill=None,
        mode=None,
        line_color='lightgray', showlegend=False
        ))

    fig_list[j].add_trace(go.Scatter(x=months, y=data['Min'],
        fill='tonexty', # fill area between trace0 and trace1
        mode=None, line_color='lightgray', showlegend=False))

    fig_list[j].add_trace(go.Scatter(x=months, y=data['5 Year Avg'], name='5 Year Avg',
                                 line=dict(color='black', width=4, dash='dot')))

    for i in [4,5]:
        fig_list[j].add_trace(go.Scatter(x=months, y=data.iloc[:,i], name=data.columns[i],
                                 line=dict(color=colors[i], width=4)))

    fig_list[j].update_layout(title=title_list[j],
                       xaxis_title='Month',
                       yaxis_title='Thousand Barrels/Day', template='plotly_white')
    
# Show all the graphs
p1_fig.show()
p2_fig.show()
p3_fig.show()
p4_fig.show()
p5_fig.show()
total_fig.show()

In [ ]:
# Links I Used
# COLOR LIST: https://community.plotly.com/t/plotly-colours-list/11730/3
# FILL IN AREA: https://plotly.com/python/filled-area-plots/
# LINE CHARTS: https://plotly.com/python/line-charts/
# PIVOT TABLES: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html